In [ ]:
import librosa
import matplotlib.pyplot as plt 
import librosa.display as libDisp
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import pandas as pd
import numpy as np
import joblib
from sklearn import svm

# SVM Model
svm_model = svm.SVC(gamma=10, C=1)

In [ ]:
notes = []
for i in range(5, 7):
    for note in ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']:
        notes.append(f'{note}{i}')

mapping = {i: notes[i] for i in range(len(notes))}

In [ ]:
def run_note(X, note_no, y):
  X_transformed = None
  targets = None
  note_name = mapping[note_no]
  targets = y[:, note_no]
  
  if len(np.unique(targets)) < 2:
    return X_transformed, targets

  lda = LinearDiscriminantAnalysis(n_components=1)
  X_transformed  = lda.fit_transform(X, targets)

  return X_transformed, targets

In [ ]:
# Train Model
train_acc = []
train_idx = ['1', '2', '4', '6', '7', '9']
for idx in train_idx:
  file_name = 'song' + idx
  a = np.load(file_name + '.npz', allow_pickle=True)

  b = a['arr_0']
  b = b.sum(axis=0)

  X = b['X']
  y = b['y']

  for i in range(len(notes)):
    X_t, y_cur = run_note(X, i, y)
    if np.any(X_t) != None:
      svm_model.fit(X_t, y_cur) #Trains the model
      acc = svm_model.score(X_t, y_cur) 
      train_acc.append(acc)

print('Done Training...')
print('Accuracy on Test Set', np.mean(train_acc))

Done Training...


In [ ]:
'''
  Save model
'''
filename = 'LDA_svm_classification.sav'
joblib.dump(svm_model, filename)

['LDA_svm_classification.sav']

In [ ]:
'''
  Valid Data
'''
valid_acc = []
loaded_model = joblib.load('LDA_svm_classification.sav')

def run_note(X, note_no, y):
  X_transformed = None
  targets = None
  note_name = mapping[note_no]
  targets = y[:, note_no]
  
  if len(np.unique(targets)) < 2:
    return X_transformed, targets

  lda = LinearDiscriminantAnalysis(n_components=1)
  X_transformed  = lda.fit_transform(X, targets)

  return X_transformed, targets


valid_idx = ['8', '10']
for idx in valid_idx:

  file_name = 'song' + idx
  a = np.load(file_name + '.npz', allow_pickle=True)

  b = a['arr_0']
  b = b.sum(axis=0)

  X = b['X']
  y = b['y']


  # Test
  for i in range(len(notes)):
    X_t, y_cur = run_note(X, i, y)
    if np.any(X_t) != None:
      X_test = X_t[:,0].reshape((X_t[:,0].shape[0], 1))
      pred = loaded_model.predict(X_test)
      acc = loaded_model.score(X_test, y_cur) 
      valid_acc.append(acc)


print('Accuracy on Validation Set', np.mean(valid_acc))

0.9425226298365071


In [ ]:
'''
  Test Data
'''

test_acc = []
loaded_model = joblib.load('LDA_svm_classification.sav')

def run_note(X, note_no, y):
  X_transformed = None
  targets = None
  note_name = mapping[note_no]
  targets = y[:, note_no]
  
  if len(np.unique(targets)) < 2:
    # print(f"note {note_name} has no samples")
    return X_transformed, targets

  lda = LinearDiscriminantAnalysis(n_components=1)
  X_transformed  = lda.fit_transform(X, targets)

  return X_transformed, targets


test_idx = ['3', '5']
for idx in test_idx:

  file_name = 'song' + idx
  a = np.load(file_name + '.npz', allow_pickle=True)

  b = a['arr_0']
  b = b.sum(axis=0)

  X = b['X']
  y = b['y']


  # Test
  for i in range(len(notes)):
    X_t, y_cur = run_note(X, i, y)
    if np.any(X_t) != None:
      X_test = X_t[:,0].reshape((X_t[:,0].shape[0], 1))
      pred = loaded_model.predict(X_test)
      acc = loaded_model.score(X_test, y_cur) #Calculates accuracy on test note
      test_acc.append(acc)


print('Accuracy on Validation Set', np.mean(test_acc))

0.910951549772559
